# About Florida Insurance Analysis

Book combines historical publicly reported data from the FL department of insurance to evaluate trends in market share, claims frequency/severity, and hurricane losses.

Data downloaded from https://apps.fldfs.com/QSRNG/Reports/ReportCriteria.aspx on 04 and 05 April 2025.

Environment AKStandard (Python 3.11.9)

# Imports

In [ ]:
import numpy as np

In [48]:
import pandas as pd # data ingestion, manipulation
import os # importing raw data - may need to just import full os
import time # I just want to very briefly look at data and have it print for a second to glance at it
from IPython.display import clear_output # thanks to ChatGPT - lets me clear printed data

# Data Extraction/Ingestion

Outline/Plan

function - import, remove grand total\
function - check/remove blank columns\
compile to a single df\
\
future: move to AW S3 bucket, SQL access

In [49]:
# listdir('./data/raw')[-1]

In [50]:
info = pd.read_excel('./data/raw/2009Q1.xlsx', skiprows = 14)
print(f"The imported shape is: {info.shape}\nThe last named insurers are:\n{info.iloc[-2:,1]}\n")
grand_total_row = info.shape[0] - 1
info.drop(grand_total_row, axis=0, inplace=True)
print(f"The updated shape is: {info.shape}\nThe last named insurers are:\n{info.iloc[-2:,1]}")

The imported shape is: (214, 65)
The last named insurers are:
212    TESLA PROPERTY & CASUALTY, INC.
213                                NaN
Name: Insurer Name, dtype: object

The updated shape is: (213, 65)
The last named insurers are:
211    STATE NATIONAL INSURANCE COMPANY INC.
212          TESLA PROPERTY & CASUALTY, INC.
Name: Insurer Name, dtype: object


# Initial EDA

In [51]:
len(info.columns)

65

In [52]:
counter = 0
for x in info.columns:
    counter += 1
    clear_output(wait = True)
    print(f"Iteration: {counter} Column: {x}\n", info[x])
    time.sleep(0.2)

Iteration: 65 Column: Unnamed: 64
 0      0%
1      0%
2      0%
3      0%
4      0%
       ..
208    0%
209    0%
210    0%
211    0%
212    0%
Name: Unnamed: 64, Length: 213, dtype: object


In [53]:
info.iloc[:,62]

0      0%
1      0%
2      0%
3      0%
4      0%
       ..
208    0%
209    0%
210    0%
211    0%
212    0%
Name: Unnamed: 62, Length: 213, dtype: object

In [54]:
info.iloc[:,62].isnull().all()

np.False_

In [55]:
x = "Rank"
# info.loc[:,x]
info.loc[:,x].isnull().all()

np.False_

In [56]:
null_columns = []
not_null_columns = []
leftover_columns = []

for x in info.columns:
    if "Unnamed" not in x and info.loc[:,x].isnull().all() == False:
        not_null_columns.append(x)
    elif "Unnamed" in x and info.loc[:,x].isnull().all() == False:
        leftover_columns.append(x)
    else:
        if info.loc[:,x].isnull().all():
            null_columns.append(x)

null_columns
not_null_columns
leftover_columns

['Unnamed: 10',
 'Unnamed: 14',
 'Unnamed: 19',
 'Unnamed: 21',
 'Unnamed: 23',
 'Unnamed: 25',
 'Unnamed: 27',
 'Unnamed: 29',
 'Unnamed: 31',
 'Unnamed: 33',
 'Unnamed: 35',
 'Unnamed: 37',
 'Unnamed: 39',
 'Unnamed: 41',
 'Unnamed: 43',
 'Unnamed: 46',
 'Unnamed: 48',
 'Unnamed: 50',
 'Unnamed: 52',
 'Unnamed: 54',
 'Unnamed: 56',
 'Unnamed: 58',
 'Unnamed: 60',
 'Unnamed: 62',
 'Unnamed: 64']

In [57]:
print(null_columns)
print(not_null_columns)
print(leftover_columns)

['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 9', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 15', 'Unnamed: 17', 'Unnamed: 18', 'Claims Opened', 'Unnamed: 45', 'Claims Closed', 'Claims Pending', 'Claims Alternative Dispute', 'Claims with Mediation', 'Claims with Arbitration', 'Claims with Appraisal', 'Claims with Sink Hole Eval', 'Claims with Settlement Conf', 'Claims with Other']
['Rank', 'Insurer Name', 'Policies In Force', 'Total $ value of exposure for policies in force that include wind coverage', 'Number of policies canceled', 'Number of policies nonrenewed', 'Number of policies canceled due to hurricane risk', 'Number of policies nonrenewed due to hurricane risk', 'Number of new policies written', 'Policies in force that exclude wind coverage', 'Total premiums written', 'Policies in force that include wind coverage', 'Total $ value of exposure for policies in force that exclude wind coverage', 'Direct premium written for policies in 

In [58]:
info[null_columns]

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 9,Unnamed: 12,Unnamed: 13,Unnamed: 15,...,Unnamed: 45,Claims Closed,Claims Pending,Claims Alternative Dispute,Claims with Mediation,Claims with Arbitration,Claims with Appraisal,Claims with Sink Hole Eval,Claims with Settlement Conf,Claims with Other
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# pandas display options - https://pandas.pydata.org/docs/user_guide/options.html

# pd.options.display.max_rows
pd.options.display.max_columns = 50
# pd.reset_option("display.max_columns")

In [60]:
mod_df = info.copy().drop(labels=null_columns, axis = 1)

mod_df.shape

(213, 42)

In [78]:
display(mod_df)

,Rank,Insurer Name,Policies In Force,Unnamed: 10,Total $ value of exposure for policies in force that include wind coverage,Unnamed: 14,Number of policies canceled,Unnamed: 19,Number of policies nonrenewed,Unnamed: 21,...,Unnamed: 46,Unnamed: 48,Unnamed: 50,Unnamed: 52,Unnamed: 54,Unnamed: 56,Unnamed: 58,Unnamed: 60,Unnamed: 62,Unnamed: 64
0,1,CITIZENS PROPERTY INSURANCE CORPORATION,1005077,16.20%,3.722960e+11,17.68%,46264,17.88%,68660,41.17%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
1,2,STATE FARM FLORIDA INSURANCE COMPANY,853867,13.77%,3.630460e+11,17.24%,42925,16.59%,12090,7.25%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
2,3,UNIVERSAL PROPERTY & CASUALTY INSURANCE COMPANY,498139,8.03%,1.062854e+11,5.05%,11644,4.50%,15616,9.36%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
3,4,"ST. JOHNS INSURANCE COMPANY, INC.",204409,3.30%,7.666128e+10,3.64%,8678,3.35%,150,0.09%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
4,5,UNITED SERVICES AUTOMOBILE ASSOCIATION,163850,2.64%,6.551545e+10,3.11%,3040,1.17%,983,0.59%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,209,AMERICAN COMMERCE INSURANCE COMPANY,0,0%,0.000000e+00,0%,1,0%,0,0%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
209,210,AMERICAN GENERAL PROPERTY INS CO OF FLORIDA,0,0%,0.000000e+00,0%,2,0%,0,0%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
210,211,EMC PROPERTY & CASUALTY COMPANY,0,0%,0.000000e+00,0%,2,0%,0,0%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
211,212,STATE NATIONAL INSURANCE COMPANY INC.,0,0%,0.000000e+00,0%,0,0%,0,0%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%


# Comparing Raw Data

I need to figure out if all of the raw data files have the same width/number of columns and if the same columns are blank between them.

In [63]:
file_names = [file for file in os.listdir('./data/raw') if file[0] != '.']

raw_data_detail = {}
for file_name in file_names:
    raw_data = pd.read_excel(f'./data/raw/{file_name}', skiprows = 14)
    width = len(raw_data.columns)
    column_names = list(raw_data.columns)
    full_columns = {}
    for column in raw_data.columns:
        if raw_data.loc[:,column].isnull().all() == False:
            full_columns.update({column : 1})
        else:
            full_columns.update({column : 0})
    raw_data_detail.update({file_name[:6] : {'width' : width, 'full_columns' : full_columns}}) #  'column_names' : column_names, unnecessary bc all column names in "full_columns"

individual_width = []
for file in raw_data_detail.keys():
    individual_width.append((raw_data_detail[file]['width']))
set(individual_width)

{65}

In [64]:
raw_data_detail['2009Q4'] #['full_columns']

{'width': 65,
 'full_columns': {'Rank': 1,
  'Insurer Name': 1,
  'Unnamed: 2': 0,
  'Unnamed: 3': 0,
  'Unnamed: 4': 0,
  'Unnamed: 5': 0,
  'Unnamed: 6': 0,
  'Unnamed: 7': 0,
  'Policies In Force': 1,
  'Unnamed: 9': 0,
  'Unnamed: 10': 1,
  'Total $ value of exposure for policies in force that include wind coverage': 1,
  'Unnamed: 12': 0,
  'Unnamed: 13': 0,
  'Unnamed: 14': 1,
  'Unnamed: 15': 0,
  'Number of policies canceled': 1,
  'Unnamed: 17': 0,
  'Unnamed: 18': 0,
  'Unnamed: 19': 1,
  'Number of policies nonrenewed': 1,
  'Unnamed: 21': 1,
  'Number of policies canceled due to hurricane risk': 1,
  'Unnamed: 23': 1,
  'Number of policies nonrenewed due to hurricane risk': 1,
  'Unnamed: 25': 1,
  'Number of new policies written': 1,
  'Unnamed: 27': 1,
  'Policies in force that exclude wind coverage': 1,
  'Unnamed: 29': 1,
  'Total premiums written': 1,
  'Unnamed: 31': 1,
  'Policies in force that include wind coverage': 1,
  'Unnamed: 33': 1,
  'Total $ value of exposu

In [65]:
# confirm all columns match either having data or being blank
check_columns = pd.DataFrame.from_dict({file_name: values["full_columns"] for file_name, values in raw_data_detail.items()}, orient="index")

for column in check_columns.columns:
    if raw_data.loc[:,column].isnull().all() == True:
        check_columns.drop(column, axis = 1, inplace = True)
check_columns.sort_index(inplace=True)
pd.set_option("display.max_columns", None)
display(check_columns.head(50))
pd.reset_option("display.max_columns")

,Rank,Insurer Name,Policies In Force,Unnamed: 10,Total $ value of exposure for policies in force that include wind coverage,Unnamed: 14,Number of policies canceled,Unnamed: 19,Number of policies nonrenewed,Unnamed: 21,Number of policies canceled due to hurricane risk,Unnamed: 23,Number of policies nonrenewed due to hurricane risk,Unnamed: 25,Number of new policies written,Unnamed: 27,Policies in force that exclude wind coverage,Unnamed: 29,Total premiums written,Unnamed: 31,Policies in force that include wind coverage,Unnamed: 33,Total $ value of exposure for policies in force that exclude wind coverage,Unnamed: 35,Direct premium written for policies in force that include wind coverage,Unnamed: 37,Direct premium written for policies in force that exclude wind coverage,Unnamed: 39,Number of policies transferred to other insurers,Unnamed: 41,Number of policies received from other insurers,Unnamed: 43,Claims Opened,Unnamed: 46,Claims Closed,Unnamed: 48,Claims Pending,Unnamed: 50,Claims Alternative Dispute,Unnamed: 52,Claims with Mediation,Unnamed: 54,Claims with Arbitration,Unnamed: 56,Claims with Appraisal,Unnamed: 58,Claims with Sink Hole Eval,Unnamed: 60,Claims with Settlement Conf,Unnamed: 62,Claims with Other,Unnamed: 64
2009Q1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1
2009Q2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1
2009Q3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1
2009Q4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1
2010Q1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1
2010Q2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1
2010Q3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1
2010Q4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1
2011Q1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1
2011Q2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1


By visual inspection, it looks like 2019Q3 is the first time data were reported for Claims Open and after.

In [66]:
d2023Q2 = pd.read_excel(f'./data/raw/2023Q2.xlsx', skiprows = 14)
# d2023Q2df = d2023Q2.loc[:,'Claims with Arbitration']
col_loc_23 = d2023Q2.columns.get_loc("Claims with Arbitration")
d2009Q3 = pd.read_excel(f'./data/raw/2009Q3.xlsx', skiprows = 14)
# d2009Q3df = d2009Q3.loc[:,'Claims with Arbitration']
col_loc_09 = d2009Q3.columns.get_loc("Claims with Arbitration")

In [67]:
selected_cols = d2023Q2.iloc[:, col_loc_23-2:col_loc_23+3]
selected_cols

,Claims with Mediation,Unnamed: 54,Claims with Arbitration,Unnamed: 56,Claims with Appraisal
0,720,78.60%,0,0%,3803
1,0,0%,0,0%,0
2,68,7.42%,333,80.43%,169
3,0,0%,0,0%,0
4,0,0%,0,0%,0
...,...,...,...,...,...
99,1,0.11%,0,0%,0
100,0,0%,0,0%,0
101,0,0%,0,0%,0
102,0,0%,0,0%,0


In [68]:
selected_cols = d2009Q3.iloc[:, col_loc_09-2:col_loc_09+3]
selected_cols

,Claims with Mediation,Unnamed: 54,Claims with Arbitration,Unnamed: 56,Claims with Appraisal
0,NaN,0%,NaN,0%,NaN
1,NaN,0%,NaN,0%,NaN
2,NaN,0%,NaN,0%,NaN
3,NaN,0%,NaN,0%,NaN
4,NaN,0%,NaN,0%,NaN
...,...,...,...,...,...
207,NaN,0%,NaN,0%,NaN
208,NaN,0%,NaN,0%,NaN
209,NaN,0%,NaN,0%,NaN
210,NaN,0%,NaN,0%,NaN


In [69]:
# Find the last valid column before "claims opened" to keep data consistent between pre2019Q3 data when removing\
# invalid columns that are caused by importing from Excel
last_valid_column = d2023Q2.columns.get_loc('Unnamed: 43')
last_valid_column

43

So I need to take all columns before 43 that are unlabeled and have no data in them and remove them. Each column with a label will be number of policies in that category; to the right of each column with values will be it's percentage. After column 43, for data pre2019Q3, the labeled columns will be empty/NA but should not be deleted; and the columns to their right will be 0%s.

Alternatively, keep all labeled columns; where unnamed AND NA data, drop. Then rename columns to the right of labeled as % of preceding column

In [70]:
# Let's just look at d2009Q3 first.
display(d2009Q3)

for column in d2009Q3.columns:
    if "Unnamed" in column and raw_data.loc[:,column].isnull().all() == True:
        # print(f"{column} is unnamed.")
        d2009Q3.drop(column, axis = 1, inplace = True)
    else:
        # print(f"{column} is NOT unnamed.")
        pass

display(d2009Q3)

,Rank,Insurer Name,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Policies In Force,Unnamed: 9,...,Claims with Arbitration,Unnamed: 56,Claims with Appraisal,Unnamed: 58,Claims with Sink Hole Eval,Unnamed: 60,Claims with Settlement Conf,Unnamed: 62,Claims with Other,Unnamed: 64
0,1,CITIZENS PROPERTY INSURANCE CORPORATION,NaN,NaN,NaN,NaN,NaN,NaN,1031546,NaN,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
1,2,STATE FARM FLORIDA INSURANCE COMPANY,NaN,NaN,NaN,NaN,NaN,NaN,790385,NaN,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
2,3,UNIVERSAL PROPERTY & CASUALTY INSURANCE COMPANY,NaN,NaN,NaN,NaN,NaN,NaN,532874,NaN,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
3,4,"ST. JOHNS INSURANCE COMPANY, INC.",NaN,NaN,NaN,NaN,NaN,NaN,198727,NaN,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
4,5,UNITED SERVICES AUTOMOBILE ASSOCIATION,NaN,NaN,NaN,NaN,NaN,NaN,159748,NaN,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,208,COLONIAL AMERICAN CASUALTY AND SURETY COMPANY,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
208,209,LIBERTY AMERICAN INSURANCE COMPANY,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
209,210,LIBERTY AMERICAN SELECT INSURANCE COMPANY,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
210,211,"USIC OF FLORIDA, INC.",NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%


,Rank,Insurer Name,Policies In Force,Unnamed: 10,Total $ value of exposure for policies in force that include wind coverage,Unnamed: 14,Number of policies canceled,Unnamed: 19,Number of policies nonrenewed,Unnamed: 21,...,Claims with Arbitration,Unnamed: 56,Claims with Appraisal,Unnamed: 58,Claims with Sink Hole Eval,Unnamed: 60,Claims with Settlement Conf,Unnamed: 62,Claims with Other,Unnamed: 64
0,1,CITIZENS PROPERTY INSURANCE CORPORATION,1031546,16.64%,3.856419e+11,17.89%,35875,15.52%,106337,57.25%,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
1,2,STATE FARM FLORIDA INSURANCE COMPANY,790385,12.75%,3.461672e+11,16.06%,28019,12.12%,3218,1.73%,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
2,3,UNIVERSAL PROPERTY & CASUALTY INSURANCE COMPANY,532874,8.59%,1.135316e+11,5.27%,12188,5.27%,15951,8.59%,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
3,4,"ST. JOHNS INSURANCE COMPANY, INC.",198727,3.21%,7.528658e+10,3.49%,10481,4.53%,242,0.13%,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
4,5,UNITED SERVICES AUTOMOBILE ASSOCIATION,159748,2.58%,6.729733e+10,3.12%,4529,1.96%,857,0.46%,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,208,COLONIAL AMERICAN CASUALTY AND SURETY COMPANY,0,0%,0.000000e+00,0%,0,0%,4,0%,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
208,209,LIBERTY AMERICAN INSURANCE COMPANY,0,0%,0.000000e+00,0%,54,0.02%,174,0.09%,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
209,210,LIBERTY AMERICAN SELECT INSURANCE COMPANY,0,0%,0.000000e+00,0%,27,0.01%,107,0.06%,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
210,211,"USIC OF FLORIDA, INC.",0,0%,0.000000e+00,0%,27,0.01%,63,0.03%,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%


The simplified analysis appears much mor efficient and valid comparing dataframe to excel data. Will update all data in this manner, and correct "unnamed" columns as percentages. I think it wold be best to collect all data in a single dataframe, so maybe it would be easier to only correct the titles on a master dataframe and only append subsequent data to that dataframe? I should also remove the "grand total" row in doing this because it will be redundant/unnecessary while I can calculate this value PRN.

In [71]:
# remove "grand total" row, rename columns to demonstrate doing so.

# d2009Q3.drop(index = d2009Q3.index[-1],inplace=True)

for column in d2009Q3.columns:
    if "Unnamed" in column:
        current_column_number = d2009Q3.columns.get_loc(column)
        name_preceding = d2009Q3.columns[current_column_number - 1]
        new_column_name = "Percentage " + name_preceding
        d2009Q3.rename(columns = {column : new_column_name}, inplace = True)

In [72]:
file_names.sort()
file_names[0:8]

['2009Q1.xlsx',
 '2009Q2.xlsx',
 '2009Q3.xlsx',
 '2009Q4.xlsx',
 '2010Q1.xlsx',
 '2010Q2.xlsx',
 '2010Q3.xlsx',
 '2010Q4.xlsx']

In [73]:
# create fresh df, take column labels from first ordered dataset, add column to specify date, clean dataset, add to master df

file_names = [file for file in os.listdir('./data/raw') if file[0] != '.']

file_names.sort()

# function to simplify and remove extra columns
def remove_empty_columns(processing_df):
    for column in processing_df.columns:
            if "Unnamed" in column and processing_df.loc[:,column].isnull().all() == True:
                processing_df.drop(column, axis = 1, inplace = True)
    # remove the "grand total" row
    processing_df.drop(index = processing_df.index[-1],inplace=True)
    return processing_df

# import only the first file to begin the dataframe
master_df = pd.read_excel(f'./data/raw/{file_names[0]}', skiprows = 14)

remove_empty_columns(master_df)

# rename columns in master
for column in master_df.columns:
    if "Unnamed" in column:
        current_column_number = master_df.columns.get_loc(column)
        name_preceding = master_df.columns[current_column_number - 1]
        new_column_name = "Percentage " + name_preceding
        master_df.rename(columns = {column : new_column_name}, inplace = True)

# add column to identify which dataset the data come from
master_df.insert(0, 'dataset', value = file_names[0][:6])

# import remaining datasets
for file_name in file_names[1:]:
    working_df = pd.read_excel(f'./data/raw/{file_name}', skiprows = 14)
    remove_empty_columns(working_df)
    working_df.insert(0, 'dataset', value = file_name[:6])
    master_df = pd.concat([master_df, working_df], axis = 0, ignore_index = True) #ChatGPT assures me this is unlikely to be a problem to keep reiterating over the same df

master_df

,dataset,Rank,Insurer Name,Policies In Force,Percentage Policies In Force,Total $ value of exposure for policies in force that include wind coverage,Percentage Total $ value of exposure for policies in force that include wind coverage,Number of policies canceled,Percentage Number of policies canceled,Number of policies nonrenewed,...,Unnamed: 46,Unnamed: 48,Unnamed: 50,Unnamed: 52,Unnamed: 54,Unnamed: 56,Unnamed: 58,Unnamed: 60,Unnamed: 62,Unnamed: 64
0,2009Q1,1,CITIZENS PROPERTY INSURANCE CORPORATION,1005077,16.20%,3.722960e+11,17.68%,46264,17.88%,68660,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009Q1,2,STATE FARM FLORIDA INSURANCE COMPANY,853867,13.77%,3.630460e+11,17.24%,42925,16.59%,12090,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009Q1,3,UNIVERSAL PROPERTY & CASUALTY INSURANCE COMPANY,498139,8.03%,1.062854e+11,5.05%,11644,4.50%,15616,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009Q1,4,"ST. JOHNS INSURANCE COMPANY, INC.",204409,3.30%,7.666128e+10,3.64%,8678,3.35%,150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009Q1,5,UNITED SERVICES AUTOMOBILE ASSOCIATION,163850,2.64%,6.551545e+10,3.11%,3040,1.17%,983,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10433,2024Q4,81,FIDELITY AND DEPOSIT COMPANY OF MARYLAND,0,NaN,0.000000e+00,NaN,0,NaN,0,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
10434,2024Q4,82,MAIN STREET AMERICA PROTECTION INSURANCE COMPANY,0,NaN,0.000000e+00,NaN,0,NaN,1,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
10435,2024Q4,83,NATIONWIDE INSURANCE COMPANY OF FLORIDA,0,NaN,0.000000e+00,NaN,0,NaN,0,...,0.09%,0%,0.09%,0%,0%,0%,0%,0%,0%,0%
10436,2024Q4,84,PATRIOT SELECT PROPERTY AND CASUALTY INSURANCE...,0,NaN,0.000000e+00,NaN,0,NaN,0,...,0.01%,0%,0.01%,0%,0%,0%,0%,0%,0%,0%


The output still has some NA columns being appended? Not sure which dataset they come from.

In [ ]:
# check if NaN columns are truly empty or if they have something in them. Why don't they have names????

for x in master_df.index:
    if pd.notna(master_df.loc[x,'Unnamed: 64']) and master_df.loc[x,'Unnamed: 64'] != '0%':
        print(master_df.loc[x,'Unnamed: 64'])

99.44%
0.56%
100%
78.87%
18.20%
2.93%
97.31%
2.15%
0.55%
90.82%
7.69%
1.48%
88.98%
8.95%
2.06%
86.82%
10.01%
0.11%
2.84%
0.11%
0.11%
92.14%
6.59%
1.26%


# Testing

In [74]:
test = 'this is a test string thingy line'
if "hi" in test:
    print('hi')

hi


In [75]:
import time

items = ["Item 1", "Item 2", "Item 3", "Item 4"]

for item in items:
    clear_output(wait = True)
    print(item)
    time.sleep(1)

# Optionally move to a new line after the loop
print()


Item 4



In [76]:
# chatty G getting wild

from IPython.display import display, clear_output
import ipywidgets as widgets

items = ["Apple", "Banana", "Cherry", "Date"]
saved_items = []
index = 0  # Keeps track of which item we're on

# Buttons
save_button = widgets.Button(description="Save")
skip_button = widgets.Button(description="Skip")
output = widgets.Output()

def handle_click(button):
    global index
    if button == save_button:
        saved_items.append(items[index])
    
    index += 1
    if index < len(items):
        show_item()
    else:
        clear_output()
        print("Done! Saved items:", saved_items)

def show_item():
    clear_output(wait=True)
    print("Current item:", items[index])
    display(save_button, skip_button)

save_button.on_click(handle_click)
skip_button.on_click(handle_click)

# Start it off
show_item()


Current item: Apple


Button(description='Save', style=ButtonStyle())

Button(description='Skip', style=ButtonStyle())